# App: WhatsApp.ipynb

- Desenvolvedor: ***Cazaroli***
- Data Início: 22/01/2021
- Atualizado:  **25/01/2021**

- O computador deve ter instalado:
    - Python 3 (melhor instalar o Anaconda, pois já instala o Python, as bibliotecas mais usadas e o Jupter)
    - Sqlite3  (resolvi usar como arquivo externo, citado nas especificações, um arquivo de Banco de Dados; poderia ser txt, Excel, etc)
    - Jupter notebbook
    - GoogleChrome 
    - e as bibliotecas citadas abaixo

- Bibliotecas instaladas:
    - pip3 install selenium
    - pip3 install webdriver_manager

- Arquivo Chromedriver:
    - deve ser baixado no site do Goggle em: https://chromedriver.chromium.org
    - precisa estar no mesmo diretório que o do App

- App: **WhatsApp.ipynb**
    - O App foi desenvolvido aqui no Jupter Notebook por facilidade de testes
    - para torná-lo um App independente do Jupter é só ir no menu File/Download as/Python(.py)
    - caso não fosse apenas um teste e sim um App de uso final, o certo seria gerar um executável para ficar transparente para o usuário

- Para executar o App
    - rodar primeiro a célula do Jupter que contem a criação do Banco de Dados chamado Dados.db
    - a tabela Contatos será criada e colocado alguns dados para teste:
        - há dois registros com dados certo que serão enviadas mensagens e
        - os outros com cada tipo de erro possível que foi indicado nas especificações
    - depois rode a célula principal, onde contém o App própriamente dito
    
- App em execução
    - será carregado uma instância do GoogleChrome
    - a página do WhatsApp Web será carregada
    - prepare seu celular nas configurações do WhatsApp para escanear o QR Code (única intervenção manual)
    - as mensagens serão enviadas
    - o GoogleChrome será fechado

- Observação:
    - o App foi desenvolvido e testado em um computador **Mac**
    - no Windows tudo deve funcionar corretamente também

In [201]:
# Geração dos Dados de Entrada

import sqlite3
import datetime

try:
    con = sqlite3.connect('Dados.db')
    cur = con.cursor()

    #criando Tabela de Contatos; poderia se estudar a possibilidade de uma tabela de Envios
    criaTabela = 'CREATE TABLE IF NOT EXISTS Contatos (idMensagem integer primary key autoincrement not null,'\
                    'DDD varchar(2),celular varchar(10),operadora varchar(10),horario_envio varchar(8),'\
                    'mensagem varchar(140))'

    cur.execute(criaTabela)

    #gravando Dados de Contatos para Teste
    horario = str(datetime.datetime.now())
    mensagem = 'sapien Mensagem' + 141*'-'
    x = 'INSERT INTO Contatos (DDD,celular,operadora,horario_envio,mensagem) values (?,?,?,?,?)'
    dados = [('35','99733-2055','VIVO',horario,'sapien Correto 1'),
            ('11','98894-5852','VIVO',horario,'sapien SP'),
            ('29','99733-2055','VIVO',horario,'sapien DDD'),
            ('35','79733-2055','VIVO',horario,'sapien Primeiro Dígito'),
            ('35','95733-2055','VIVO',horario,'sapien Segundo Dígito'),
            ('35','99733-2055','VIVO','2021-01-23 20:00:00','sapien Horário'),
            ('35','99733-2055','VIVO',horario,mensagem),
            ('35','99733-2055','VIVO',horario,'sapien Correto 2')]

    for reg in dados:
        cur.execute(x,reg)

    con.commit()

    cur.close()
    con.close()

except Exception as e:
    print('Erro:', e)

In [203]:
# App: WhatsApp

# importação das bibliotecas
from selenium import webdriver                            # controle do navegador através do Python
#from webdriver_manager.chrome import ChromeDriverManager # navegador GoogleChrome
from selenium.webdriver.common.keys import Keys           # ter funcionalidade do teclado no navegador
import sqlite3       # Banco de Dados usado p armazenar os contatos
import re            # expressões regulares RegEx
import requests      # obtendo dados da internet p verificar balckList
import os            # funções do Sistema Operacional
import time          # função Sleep para dar tempo
from datetime import datetime # funções de data e hora

class WhatsAppBot:
    def __init__(self): # preparação do WebDriver para o WhatsApp / Definição Dados
        op = webdriver.ChromeOptions()
        op.add_argument('lang=pt-br')
        
        caminho = os.getcwd() # diretório do App
        # o chromedriver precisa estar no mesmo diretório que o App
        self.driver = webdriver.Chrome(executable_path=caminho + r'/chromedriver') 
        
        # outra forma de fazer que não foi usada
        #self.driver = webdriver.Chrome(ChromeDriverManager().install()) 
        
    def obtemDados(self): # obtenção dos Dados
        try:
            con = sqlite3.connect('Dados.db')
            cur = con.cursor()

            #lendo Dados
            #caso possua mais de uma mensagem para o mesmo destino,
            #apenas a mensagem apta com o menor horário deve ser considerada
            x = 'SELECT DISTINCT * FROM Contatos ORDER BY ddd, celular, mensagem, horario_envio'
            cur.execute(x)
            dadosLidos = cur.fetchall()
            
            cur.close()
            con.close()
        
            return dadosLidos
        except Exception as e:
            print('Erro:', e)
    
    def verificaDados(self, ddd, celular, operadora, horario, mensagem):
        #mensagens com telefone inválido deverão ser bloqueadas(DDD+NUMERO)
        #mensagens para o estado de São Paulo deverão ser bloqueadas
        flagF = self.validaTelefone(ddd, celular)
        
        #mensagens com agendamento após as 19:59:59 deverão ser bloqueadas
        if horario <= datetime.strptime('19:59:59', '%H:%M:%S'):
            flagH = True
        else:
            flagH = False  
            
        #mensagens com mais de 140 caracteres deverão ser bloqueadas
        if len(mensagem) <= 140:
            flagM = True
        else:
            flagM = False
            
        #mensagens que estão na blacklist deverão ser bloqueadas
            # https://front-test-pg.herokuapp.com/blacklist/:phone
                # Se retornar 200, está na blacklist
                # Se retornar 404 não está na blacklist
        res = requests.get('https://front-test-pg.herokuapp.com/blacklist/:ddd+celular')
        
        if res.status_code == 200:
            flagBL = False # bloqueia, pois está na blackList
        else:
            flagBL = True

        #o id_broker será definido conforme a operadora
            #ID_BROKER	OPERADORAS
                #1 VIVO, TIM
                #2 CLARO, OI
                #3 NEXTEL
        flagBK = True # não entendi essa validação, por isso coloquei True
        
        #print('Fone',flagF,'Horário',flagH,'Mensagem',flagM,'BlackList',flagBL,'Broken',flagBK,mensagem)
        
        if flagF and flagH and flagM and flagBL and flagBK:
          return True  
        else:
          return False
    
    def validaTelefone(self, ddd, celular):
        # DDD com 2 digitos e ser válido [14689][1-9]|2[12478]|3[1234578]|5[1345]|7[134579]
        # [14689][1-9] Todos os Estados exceto os abaixo
        # [4689][1-9] Todos os Estados exceto os abaixo e o estado de SP (1)
        # 2[12478] para Rio de Janeiro e Espírito Santo, 3[1234578] para Minas Gerais,
        # 5[1345] para Rio Grande do Sul, 7[134579] para Bahia e Sergipe
        padrao = '[4689][1-9]|2[12478]|3[1234578]|5[1345]|7[134579]'
  
        if re.match(padrao,ddd) and len(ddd) == 2:
          flagDDD = True  
        else:
          flagDDD = False

        # número celular deve conter 9 dígitos, deve começar com 9 [9]{1} e o segundo dígito deve ser > 6 [6-9]{1}
        padrao = '[9]{1}[6-9]{1}[0-9]{7}'  
        if re.match(padrao,celular) and len(celular) == 9:
          flagC = True  
        else:
          flagC = False
        
        if flagDDD and flagC:
          return True  
        else:
          return False
    
    def EnviaMensagens(self):
        self.driver.get('https://web.whatsapp.com/') 
        time.sleep(10) # dá tempo p scanear o QR Code
        
        dadosLidos = self.obtemDados()
        
        for reg in dadosLidos:
            ddd = reg[1]
            celular = reg[2].replace('-', '')
            operadora = reg[3]
            horario = datetime.strptime(reg[4][11:19], '%H:%M:%S')
            mensagem = reg[5]
            
            if self.verificaDados(ddd, celular, operadora, horario, mensagem) == False:
                continue # aqui poderia ser colocado a gravação de um log de erros
            
            x = celular.find('9') # procura o primeiro 9 e retira, pois o WhatsApp não o usa
            celular = celular[0:x] + celular[x+1:]
            destino = '+55' + ddd + celular
            
            # no Chrome p verificar os códigos HTML, clica-se no local, botão direito e Inspecionar
            
            # caixa de texto para pesquisar o contato
            txtPesquisa = self.driver.find_element_by_xpath("//div[contains(@class,'copyable-text selectable-text')]")
            txtPesquisa.click()
            txtPesquisa.send_keys(destino)
            txtPesquisa.send_keys(Keys.ENTER)
            
            # caixa de texto para escrever a mensagem (observe o s nesse comando element, ficando elements)
            txtMensagem = self.driver.find_elements_by_xpath("//div[contains(@class,'copyable-text selectable-text')]")
            txtMensagem[1].click() # tem dois campos desses, 0 e 1; pegamos o segundo
            txtMensagem[1].send_keys(mensagem)
            txtMensagem[1].send_keys(Keys.ENTER)
   
        self.driver.close() # fecha a instância do navegador Chrome; com quit() fecha todas
        
bot = WhatsAppBot()
bot.EnviaMensagens()